## Conectar al Drive para acceder a los datos

In [2]:

try:
    from google.colab import drive
    drive.mount('/content/drive')
    BASE = "/content/drive/MyDrive"
except Exception as e:
    print("No estás en Colab o ya está montado. Usa ruta local.")
    BASE = "."


Mounted at /content/drive


In [ ]:
import os, glob
import pandas as pd
import unicodedata

# Carpeta donde están los Excel
CARPETA = os.path.join(BASE, "DATA_LIMPIEZA")

# Busca sólo .xlsx (si tuvieras .xls cambia el patrón o agrega otro glob)
rutas = sorted(glob.glob(os.path.join(CARPETA, "*.xlsx")))

print(f"Encontrados {len(rutas)} archivos en: {CARPETA}")
for r in rutas[:3]:
    print("•", os.path.basename(r))


Encontrados 23 archivos en: /content/drive/MyDrive/DATA_LIMPIEZA
• CuidadCapital.xlsx
• Elprogreso.xlsx
• Guatemala.xlsx


## Normalizar los nombre de archivos

In [ ]:
def normaliza(texto: str) -> str:
    if texto is None:
        return ""
    # quita acentos
    texto = unicodedata.normalize("NFKD", texto)
    texto = "".join(ch for ch in texto if not unicodedata.combining(ch))
    # limpieza básica
    texto = texto.strip()
    texto = texto.replace("\n", " ").replace("\r", " ")
    # espacios -> guiones bajos, minúsculas
    texto = "_".join(texto.split())
    return texto.lower()


## Mapea cada uno de ellos y muestra sus encabezados, Si hay una columna vacia la elimina

In [ ]:
encabezados_por_archivo = []  # lista de dicts para DataFrame resumen

for ruta in rutas:
    nombre = os.path.basename(ruta)
    try:
        # Lee solo encabezados (nrows=0) de la PRIMERA hoja
        df0 = pd.read_excel(ruta, sheet_name=0, nrows=0, engine="openpyxl")

        # Detectar columnas Unnamed
        cols_original = list(map(str, df0.columns.tolist()))
        cols_unnamed = [c for c in cols_original if c.startswith("Unnamed")]

        if cols_unnamed:
            print(f"\n⚠ {nombre} contiene columnas vacías: {cols_unnamed}")
            # Eliminar columnas Unnamed
            df0 = df0.loc[:, ~df0.columns.str.contains('^Unnamed')]
            # Actualizar lista de columnas originales sin las vacías
            cols_original = list(map(str, df0.columns.tolist()))

        # Normalizar nombres
        cols_normal = [normaliza(c) for c in cols_original]

        print(f"\n===== {nombre} =====")
        print("Encabezados (limpios):")
        print(cols_original)

        encabezados_por_archivo.append({
            "archivo": nombre,
            "num_columnas": len(cols_original),
            "encabezados_original": cols_original,
            "encabezados_normalizado": cols_normal,
            "set_normalizado": tuple(sorted(set(cols_normal))),
        })

    except Exception as e:
        print(f"\n===== {nombre} =====")
        print("ERROR al leer:", e)



⚠ CuidadCapital.xlsx contiene columnas vacías: ['Unnamed: 1']

===== CuidadCapital.xlsx =====
Encabezados (limpios):
['CODIGO', 'DISTRITO', 'DEPARTAMENTO', 'MUNICIPIO', 'ESTABLECIMIENTO', 'DIRECCION', 'TELEFONO', 'SUPERVISOR', 'DIRECTOR', 'NIVEL', 'SECTOR', 'AREA', 'STATUS', 'MODALIDAD', 'JORNADA', 'PLAN', 'DEPARTAMENTAL']

⚠ Elprogreso.xlsx contiene columnas vacías: ['Unnamed: 1']

===== Elprogreso.xlsx =====
Encabezados (limpios):
['CODIGO', 'DISTRITO', 'DEPARTAMENTO', 'MUNICIPIO', 'ESTABLECIMIENTO', 'DIRECCION', 'TELEFONO', 'SUPERVISOR', 'DIRECTOR', 'NIVEL', 'SECTOR', 'AREA', 'STATUS', 'MODALIDAD', 'JORNADA', 'PLAN', 'DEPARTAMENTAL']

⚠ Guatemala.xlsx contiene columnas vacías: ['Unnamed: 1']

===== Guatemala.xlsx =====
Encabezados (limpios):
['CODIGO', 'DISTRITO', 'DEPARTAMENTO', 'MUNICIPIO', 'ESTABLECIMIENTO', 'DIRECCION', 'TELEFONO', 'SUPERVISOR', 'DIRECTOR', 'NIVEL', 'SECTOR', 'AREA', 'STATUS', 'MODALIDAD', 'JORNADA', 'PLAN', 'DEPARTAMENTAL']

⚠ Izabal.xlsx contiene columnas vac

## Compara si cada archivo tiene las misma columnas

In [ ]:
res = pd.DataFrame(encabezados_por_archivo)

# Encuentra el "esquema modal" (el conjunto de columnas normalizadas más frecuente)
conteo_sets = res["set_normalizado"].value_counts()
esquema_modal = conteo_sets.index[0] if not conteo_sets.empty else tuple()
print("\n\n=== Resumen de esquemas ===")
print(conteo_sets)

# Marca si cada archivo coincide con el esquema modal
res["coincide_modal"] = res["set_normalizado"].apply(lambda s: s == esquema_modal)

# Para ver rápidamente quiénes NO coinciden
no_match = res[~res["coincide_modal"]].copy()

print("\nArchivos que NO coinciden con el esquema modal:")
if no_match.empty:
    print("✅ Todos coinciden con el mismo conjunto de encabezados (normalizados).")
else:
    for _, row in no_match.iterrows():
        base = set(esquema_modal)
        actual = set(row["set_normalizado"])
        faltantes = sorted(list(base - actual))
        extras = sorted(list(actual - base))
        print(f"\n• {row['archivo']}")
        print("  - Faltan (vs modal):", faltantes if faltantes else "—")
        print("  - Extras (vs modal):", extras if extras else "—")




=== Resumen de esquemas ===
set_normalizado
(area, codigo, departamental, departamento, direccion, director, distrito, establecimiento, jornada, modalidad, municipio, nivel, plan, sector, status, supervisor, telefono)    23
Name: count, dtype: int64

Archivos que NO coinciden con el esquema modal:
✅ Todos coinciden con el mismo conjunto de encabezados (normalizados).


## Unir todo los archivos en un CSV

In [ ]:
import os, glob, unicodedata
import pandas as pd
CARPETA = os.path.join(BASE, "DATA_LIMPIEZA")
PATRON = "*.xlsx"



rutas = []
for patron in ([PATRON] if isinstance(PATRON, str) else PATRON):
    rutas.extend(glob.glob(os.path.join(CARPETA, patron)))
rutas = sorted(rutas)

print(f"Archivos encontrados: {len(rutas)}")
for r in rutas[:5]: print("•", os.path.basename(r))


meta = []
for ruta in rutas:
    nombre = os.path.basename(ruta)
    try:
        df0 = pd.read_excel(ruta, sheet_name=0, nrows=0, engine="openpyxl")
        if df0.columns.str.contains('^Unnamed').any():
            print(f"⚠ {nombre}: eliminando columnas Unnamed en encabezados")
            df0 = df0.loc[:, ~df0.columns.str.contains('^Unnamed')]
        cols = list(map(str, df0.columns.tolist()))
        cols_norm = tuple(sorted(set(normaliza(c) for c in cols)))
        meta.append({"archivo": nombre, "ruta": ruta, "cols_original": cols, "cols_norm_set": cols_norm})
    except Exception as e:
        print(f"❌ {nombre}: error leyendo encabezados -> {e}")

meta_df = pd.DataFrame(meta)
if meta_df.empty:
    raise SystemExit("No se pudieron leer encabezados de ningún archivo.")


conteo = meta_df["cols_norm_set"].value_counts()
esquema_modal = conteo.index[0]
print("\n=== Esquema modal (set normalizado) ===")
print(esquema_modal)
print("\nFrecuencias de esquemas:")
print(conteo)

modal_ruta = meta_df[meta_df["cols_norm_set"] == esquema_modal]["ruta"].iloc[0]
df_modal = pd.read_excel(modal_ruta, engine="openpyxl")
df_modal = df_modal.loc[:, ~df_modal.columns.str.contains('^Unnamed')]
orden_canonico = [c for c in df_modal.columns]  # orden y nombres "bonitos" de referencia
canonico_norm = [normaliza(c) for c in orden_canonico]


acumulados = []
resumen = []
for _, row in meta_df.iterrows():
    ruta = row["ruta"]
    nombre = row["archivo"]
    try:
        df = pd.read_excel(ruta, engine="openpyxl")
        # quitar columnas Unnamed
        if df.columns.str.contains('^Unnamed').any():
            print(f"⚠ {nombre}: quitando columnas Unnamed")
            df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

        # normalizar nombres para mapear al canónico
        col_norm_actual = [normaliza(c) for c in df.columns]

        # construir un mapping: columna_actual -> nombre_canonico (por el norm)
        mapping = {}
        for c_act, c_norm in zip(df.columns, col_norm_actual):
            if c_norm in canonico_norm:
                # usar el nombre canónico correspondiente
                idx = canonico_norm.index(c_norm)
                mapping[c_act] = orden_canonico[idx]
            else:
                # columna no está en el esquema modal; la omitimos (o podrías conservarla)
                mapping[c_act] = None

        # renombrar y filtrar solo columnas canónicas
        df = df.rename(columns={k:v for k,v in mapping.items() if v is not None})
        df = df.loc[:, [c for c in orden_canonico if c in df.columns]]



        acumulados.append(df)
        resumen.append({"archivo": nombre, "filas": len(df), "columnas": len(df.columns)})

    except Exception as e:
        print(f"❌ {nombre}: error al cargar/alinear -> {e}")

if not acumulados:
    raise SystemExit("No se logró acumular ningún DataFrame.")

full = pd.concat(acumulados, ignore_index=True)

print("\n=== Resumen de carga ===")
print(pd.DataFrame(resumen))

print("\nShape final concatenado:", full.shape)
print("Columnas finales:", list(full.columns))


csv_out = os.path.join(CARPETA, "DATA_COMPLETAcsv")
xlsx_out = os.path.join(CARPETA, "DATA_COMPLETAcsv.xlsx")

full.to_csv(csv_out, index=False, encoding="utf-8-sig")
print("✔ CSV guardado en:", csv_out)

# opcional XLSX
with pd.ExcelWriter(xlsx_out, engine="openpyxl") as w:
    full.to_excel(w, index=False, sheet_name="UNIDOS")
print("✔ XLSX guardado en:", xlsx_out)


Archivos encontrados: 23
• CuidadCapital.xlsx
• Elprogreso.xlsx
• Guatemala.xlsx
• Izabal.xlsx
• Quetzaltenango.xlsx
⚠ CuidadCapital.xlsx: eliminando columnas Unnamed en encabezados
⚠ Elprogreso.xlsx: eliminando columnas Unnamed en encabezados
⚠ Guatemala.xlsx: eliminando columnas Unnamed en encabezados
⚠ Izabal.xlsx: eliminando columnas Unnamed en encabezados
⚠ Quetzaltenango.xlsx: eliminando columnas Unnamed en encabezados
⚠ Quiche.xlsx: eliminando columnas Unnamed en encabezados
⚠ Sanmarcos.xlsx: eliminando columnas Unnamed en encabezados
⚠ Santarosa.xlsx: eliminando columnas Unnamed en encabezados
⚠ altaverapaz.xlsx: eliminando columnas Unnamed en encabezados
⚠ bajaverapaz.xlsx: eliminando columnas Unnamed en encabezados
⚠ chimaltenango.xlsx: eliminando columnas Unnamed en encabezados
⚠ chiquimula.xlsx: eliminando columnas Unnamed en encabezados
⚠ escuintla.xlsx: eliminando columnas Unnamed en encabezados
⚠ huehuetenango.xlsx: eliminando columnas Unnamed en encabezados
⚠ jalapa.xls

# Análisis del estado de los datos crudos (antes de limpiar)

En esta sección se caracteriza el estado original de los datos sin aplicar transformaciones de limpieza:
- i) inventario y esquema por archivo
- ii) valores faltantes y tipos, iii) duplicados
- iv) problemas típicos en **ESTABLECIMIENTO**, **DIRECCION** y **TELEFONO** (formato/consistencia),

- v) distribución de categorías clave.

se analiza el estado original del dataset `DATA_COMPLETA.csv` antes de realizar cualquier limpieza.
Incluye: revisión de encabezados, valores faltantes, tipos de datos, duplicados y calidad de variables críticas (`ESTABLECIMIENTO`, `DIRECCION`, `TELEFONO`).



In [8]:
import os
import re
import unicodedata
import numpy as np
import pandas as pd

OUT_DIR = os.path.join(BASE, "DATA_LIMPIEZA", "evidencias_crudo")
os.makedirs(OUT_DIR, exist_ok=True)


PATH_CSV_CRUDO = os.path.join(BASE, "DATA_LIMPIEZA", "DATA_COMPLETAcsv")


df_raw = pd.read_csv(PATH_CSV_CRUDO, dtype=str, keep_default_na=False, na_values=[""])
print(df_raw.shape)
df_raw.head(3)


(11277, 17)


,CODIGO,DISTRITO,DEPARTAMENTO,MUNICIPIO,ESTABLECIMIENTO,DIRECCION,TELEFONO,SUPERVISOR,DIRECTOR,NIVEL,SECTOR,AREA,STATUS,MODALIDAD,JORNADA,PLAN,DEPARTAMENTAL
0,00-01-0158-46,01-312,CIUDAD CAPITAL,ZONA 1,COLEGIO TECNICO PROGRESISTA CETECPRO,2A. AVENIDA 3-59,22512759,AMADO SALOMON FLORES PEREZ,IRMA AMPARO TOLEDO HERNANDEZ,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,DOBLE,DIARIO(REGULAR),GUATEMALA NORTE
1,00-01-0160-46,NaN,CIUDAD CAPITAL,ZONA 1,INSTITUTO DE EDUCACION DIVERSIFICADA 'CENTRO DE ESTUDIOS MERCADOLOGICOS Y PUBLICITARIOS',8A AVE. 3-50,22329819,NaN,NaN,DIVERSIFICADO,PRIVADO,URBANA,CERRADA DEFINITIVAMENTE,MONOLINGUE,MATUTINA,DIARIO(REGULAR),GUATEMALA NORTE
2,00-01-0162-46,01-102,CIUDAD CAPITAL,ZONA 1,INSTITUTO PRIVADO MIXTO DE EDUCACION DIVERSIFICADA CENTRO EDUCACIONAL GUATEMALTECO,11 CALLE 9-33,22381718,ZIZI ARELY LOPEZ CHINCHILLA,----,DIVERSIFICADO,PRIVADO,URBANA,CERRADA DEFINITIVAMENTE,MONOLINGUE,MATUTINA,DIARIO(REGULAR),GUATEMALA NORTE


In [9]:
# Foto de columnas crudo (orden y nombres exactos)
cols_crudo = pd.DataFrame({"columna": df_raw.columns})
cols_crudo.to_csv(os.path.join(OUT_DIR, "01_cols_crudo.csv"), index=False)

# Tipos "inferidos" suavemente (para diagnóstico, no para limpieza)
tipos = df_raw.infer_objects(copy=False).dtypes.astype(str).reset_index()
tipos.columns = ["columna", "tipo_inferido"]
tipos.to_csv(os.path.join(OUT_DIR, "02_tipos_inferidos.csv"), index=False)

print("Columnas crudo:")
display(cols_crudo)
print("\nTipos inferidos:")
display(tipos)

print("\nMuestra cruda (5 filas):")
display(df_raw.head(5))


Columnas crudo:


,columna
0,CODIGO
1,DISTRITO
2,DEPARTAMENTO
3,MUNICIPIO
4,ESTABLECIMIENTO
5,DIRECCION
6,TELEFONO
7,SUPERVISOR
8,DIRECTOR
9,NIVEL



Tipos inferidos:


,columna,tipo_inferido
0,CODIGO,object
1,DISTRITO,object
2,DEPARTAMENTO,object
3,MUNICIPIO,object
4,ESTABLECIMIENTO,object
5,DIRECCION,object
6,TELEFONO,object
7,SUPERVISOR,object
8,DIRECTOR,object
9,NIVEL,object



Muestra cruda (5 filas):


,CODIGO,DISTRITO,DEPARTAMENTO,MUNICIPIO,ESTABLECIMIENTO,DIRECCION,TELEFONO,SUPERVISOR,DIRECTOR,NIVEL,SECTOR,AREA,STATUS,MODALIDAD,JORNADA,PLAN,DEPARTAMENTAL
0,00-01-0158-46,01-312,CIUDAD CAPITAL,ZONA 1,COLEGIO TECNICO PROGRESISTA CETECPRO,2A. AVENIDA 3-59,22512759,AMADO SALOMON FLORES PEREZ,IRMA AMPARO TOLEDO HERNANDEZ,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,DOBLE,DIARIO(REGULAR),GUATEMALA NORTE
1,00-01-0160-46,NaN,CIUDAD CAPITAL,ZONA 1,INSTITUTO DE EDUCACION DIVERSIFICADA 'CENTRO DE ESTUDIOS MERCADOLOGICOS Y PUBLICITARIOS',8A AVE. 3-50,22329819,NaN,NaN,DIVERSIFICADO,PRIVADO,URBANA,CERRADA DEFINITIVAMENTE,MONOLINGUE,MATUTINA,DIARIO(REGULAR),GUATEMALA NORTE
2,00-01-0162-46,01-102,CIUDAD CAPITAL,ZONA 1,INSTITUTO PRIVADO MIXTO DE EDUCACION DIVERSIFICADA CENTRO EDUCACIONAL GUATEMALTECO,11 CALLE 9-33,22381718,ZIZI ARELY LOPEZ CHINCHILLA,----,DIVERSIFICADO,PRIVADO,URBANA,CERRADA DEFINITIVAMENTE,MONOLINGUE,MATUTINA,DIARIO(REGULAR),GUATEMALA NORTE
3,00-01-0168-46,NaN,CIUDAD CAPITAL,ZONA 1,INSTITUTO DE EDUCACION DIVERSIFICADA 'LICEO MARIANO GALVEZ',2A AVE. 9-82,NaN,NaN,NaN,DIVERSIFICADO,PRIVADO,URBANA,CERRADA DEFINITIVAMENTE,MONOLINGUE,VESPERTINA,DIARIO(REGULAR),GUATEMALA NORTE
4,00-01-0173-46,01-301,CIUDAD CAPITAL,ZONA 1,INSTITUTO NORMAL PARA SEÑORITAS CENTRO AMERICA,1A CALLE 2-64,22323424,LUCRECIA MARISOL CERMEÑO GONZÁLEZ,INGRID MARIELA ESPINA ORELLANA,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),GUATEMALA NORTE


In [10]:
faltantes = df_raw.replace({"": np.nan}).isna().sum().to_frame("faltantes")
faltantes["total"] = len(df_raw)
faltantes["porcentaje"] = (faltantes["faltantes"] / faltantes["total"] * 100).round(2)
faltantes = faltantes.sort_values("porcentaje", ascending=False).reset_index().rename(columns={"index": "columna"})
faltantes.to_csv(os.path.join(OUT_DIR, "03_faltantes_por_columna.csv"), index=False)
display(faltantes)


,columna,faltantes,total,porcentaje
0,DIRECTOR,1499,11277,13.29
1,TELEFONO,935,11277,8.29
2,SUPERVISOR,528,11277,4.68
3,DISTRITO,525,11277,4.66
4,DIRECCION,77,11277,0.68
5,ESTABLECIMIENTO,4,11277,0.04
6,CODIGO,0,11277,0.00
7,MUNICIPIO,0,11277,0.00
8,DEPARTAMENTO,0,11277,0.00
9,NIVEL,0,11277,0.00


In [11]:
# Duplicados completos
dups_full_count = df_raw.duplicated(keep=False).sum()
print(f"Duplicados completos (todas las columnas): {dups_full_count}")

dups_full_preview = df_raw[df_raw.duplicated(keep=False)].head(50)
dups_full_preview.to_csv(os.path.join(OUT_DIR, "04_duplicados_completos_preview.csv"), index=False)

# Duplicados por clave lógica
CLAVE = ["ESTABLECIMIENTO", "DIRECCION", "TELEFONO"]
claves_presentes = [c for c in CLAVE if c in df_raw.columns]
print("Clave lógica utilizada:", claves_presentes)

if claves_presentes:
    dups_key = df_raw[df_raw.duplicated(subset=claves_presentes, keep=False)]
    dups_key.to_csv(os.path.join(OUT_DIR, "05_duplicados_por_clave.csv"), index=False)
    print(f"Duplicados por clave ({claves_presentes}): {len(dups_key)}")
    display(dups_key.head(20))
else:
    print("⚠️ No se encontraron todas las columnas de la clave sugerida para revisar duplicados por clave.")


Duplicados completos (todas las columnas): 0
Clave lógica utilizada: ['ESTABLECIMIENTO', 'DIRECCION', 'TELEFONO']
Duplicados por clave (['ESTABLECIMIENTO', 'DIRECCION', 'TELEFONO']): 3009


,CODIGO,DISTRITO,DEPARTAMENTO,MUNICIPIO,ESTABLECIMIENTO,DIRECCION,TELEFONO,SUPERVISOR,DIRECTOR,NIVEL,SECTOR,AREA,STATUS,MODALIDAD,JORNADA,PLAN,DEPARTAMENTAL
18,00-01-0187-46,01-303,CIUDAD CAPITAL,ZONA 1,ESCUELA TECNICA SUPERIOR ALINARI,5A CALLE 5-61,22204274,CLAUDIA PATRICIA RUIZ CASASOLA DE ESTRADA,ANA MARIA CANEL MEDRANO,DIVERSIFICADO,PRIVADO,URBANA,CERRADA DEFINITIVAMENTE,MONOLINGUE,MATUTINA,DIARIO(REGULAR),GUATEMALA NORTE
27,00-01-0197-46,01-403,CIUDAD CAPITAL,ZONA 1,LICEO TECNOLOGICO GUATEMALA DE LA ASUNCION,3A. CALLE 7-64,54452951,CARLOS HUMBERTO GONZÁLEZ DE LEÓN,ELUBIA ANAYTE SIGUENZA MONTERROSO,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,DOBLE,FIN DE SEMANA,GUATEMALA NORTE
46,00-01-0218-46,01-403,CIUDAD CAPITAL,ZONA 1,LICEO TECNOLOGICO GUATEMALA DE LA ASUNCION,3A. CALLE 7-64,54452951,CARLOS HUMBERTO GONZÁLEZ DE LEÓN,ELUBIA ANAYTE SIGUENZA MONTERROSO,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,DOBLE,FIN DE SEMANA,GUATEMALA NORTE
76,00-01-0253-46,01-301,CIUDAD CAPITAL,ZONA 1,INSTITUTO Y ACADEMIA PRACTICA COMERCIAL,6A. CALLE 0-25,22202869,LUCRECIA MARISOL CERMEÑO GONZÁLEZ,CLAUDIA PATRICIA MANSILLA CABRERA,DIVERSIFICADO,PRIVADO,URBANA,CERRADA TEMPORALMENTE,MONOLINGUE,MATUTINA,DIARIO(REGULAR),GUATEMALA NORTE
84,00-01-0262-46,01-318,CIUDAD CAPITAL,ZONA 1,LICEO CANADIENSE,11 CALLE 1-53,23750055,LESLIE AZUCENA MONZON TECUN,ADA LUCRECIA MIJANGOS TEO,DIVERSIFICADO,PRIVADO,URBANA,CERRADA TEMPORALMENTE,MONOLINGUE,VESPERTINA,DIARIO(REGULAR),GUATEMALA NORTE
88,00-01-0266-46,01-403,CIUDAD CAPITAL,ZONA 1,LICEO DE COMPUTACION C.S.S.,10 AVENIDA 11-46,22513059,CARLOS HUMBERTO GONZÁLEZ DE LEÓN,-------------,DIVERSIFICADO,PRIVADO,URBANA,CERRADA TEMPORALMENTE,MONOLINGUE,INTERMEDIA,DIARIO(REGULAR),GUATEMALA NORTE
99,00-01-0280-46,01-305,CIUDAD CAPITAL,ZONA 1,COLEGIO TECNICO COMERCIAL NAHUALA,3A. AVENIDA 13-23,22329664,ISIS IRAZEMA CANEL PALMA,OVIDIO LISANDRA OROZCO LOPEZ,DIVERSIFICADO,PRIVADO,URBANA,CERRADA TEMPORALMENTE,MONOLINGUE,MATUTINA,DIARIO(REGULAR),GUATEMALA NORTE
103,00-01-0288-46,01-317,CIUDAD CAPITAL,ZONA 1,COLEGIO EN COMPUTACION SAN JOSE NO. 1,4A. AVENIDA 3-39,22325435,MARIANO DOMINGO ESTRADA TELETOR,ANA MARIA NORIEGA RODAS,DIVERSIFICADO,PRIVADO,URBANA,CERRADA DEFINITIVAMENTE,MONOLINGUE,MATUTINA,DIARIO(REGULAR),GUATEMALA NORTE
104,00-01-0289-46,01-317,CIUDAD CAPITAL,ZONA 1,COLEGIO EN COMPUTACION SAN JOSE NO. 1,4A. AVENIDA 3-39,22325435,MARIANO DOMINGO ESTRADA TELETOR,ANA MARIA NORIEGA RODAS,DIVERSIFICADO,PRIVADO,URBANA,CERRADA DEFINITIVAMENTE,MONOLINGUE,DOBLE,DIARIO(REGULAR),GUATEMALA NORTE
106,00-01-0293-46,01-402,CIUDAD CAPITAL,ZONA 1,COLEGIO EN COMPUTACION SAN JOSE NO. 1,4A. AVENIDA 3-39,22325435,CARLOS HUMBERTO GONZALEZ DE LEON,ANA MARIA NORIEGA RODAS,DIVERSIFICADO,PRIVADO,URBANA,CERRADA DEFINITIVAMENTE,MONOLINGUE,DOBLE,FIN DE SEMANA,GUATEMALA NORTE


In [14]:
def tiene_espacios_extras(s: pd.Series) -> pd.Series:
    # True si difiere al hacer strip o colapsar múltiples espacios
    return (s != s.str.strip()) | (s.str.replace(r"\s+", " ", regex=True) != s)

def ratio_mayusculas(s: pd.Series) -> float:
    s2 = s.fillna("")
    total = (s2.str.len()).sum()
    if total == 0: return 0.0
    mayus = s2.apply(lambda x: sum(ch.isupper() for ch in x)).sum()
    return round(mayus / total, 4)

diag_espacios = []
for col in ["ESTABLECIMIENTO","DIRECCION"]:
    if col in df_raw.columns:
        serie = df_raw[col].astype(str)
        extra = tiene_espacios_extras(serie).mean() * 100
        diag_espacios.append({"columna": col, "%_con_espacios_extras": round(extra, 2),
                              "ratio_mayusculas_en_texto": ratio_mayusculas(serie)})

diag_espacios_df = pd.DataFrame(diag_espacios)
diag_espacios_df.to_csv(os.path.join(OUT_DIR, "06_diag_espacios_casing.csv"), index=False)
display(diag_espacios_df)


,columna,%_con_espacios_extras,ratio_mayusculas_en_texto
0,ESTABLECIMIENTO,0.0,0.8743
1,DIRECCION,0.0,0.6785


In [15]:
if "TELEFONO" in df_raw.columns:
    tel = df_raw["TELEFONO"].astype(str)

    # Tel no vacíos
    tel_no_vacios = tel.replace({"": np.nan}).dropna()

    # Con caracteres no numéricos
    tel_no_numericos_mask = tel_no_vacios.str.contains(r"[^0-9]", regex=True)
    tel_no_numericos = df_raw.loc[tel_no_vacios.index[tel_no_numericos_mask]]
    tel_no_numericos.to_csv(os.path.join(OUT_DIR, "07_telefonos_no_numericos.csv"), index=False)

    # Longitud distinta a 8 (después de quitar no numéricos por diagnóstico)
    tel_norm = tel_no_vacios.str.replace(r"[^0-9]", "", regex=True)
    tel_len_distinta = df_raw.loc[tel_norm.index[(tel_norm.str.len() != 8)]]
    tel_len_distinta.to_csv(os.path.join(OUT_DIR, "08_telefonos_longitud_invalida.csv"), index=False)

    resumen_tel = pd.DataFrame({
        "total_no_vacios": [len(tel_no_vacios)],
        "con_no_numericos": [tel_no_numericos.shape[0]],
        "longitud_!=8": [tel_len_distinta.shape[0]]
    })
    resumen_tel.to_csv(os.path.join(OUT_DIR, "09_resumen_telefonos.csv"), index=False)
    display(resumen_tel)
else:
    print("⚠️ No existe columna TELEFONO en el CSV crudo.")


,total_no_vacios,con_no_numericos,longitud_!=8
0,11277,1849,1900


In [16]:
patrones = {
    "contiene_zona": r"\b(zona|zn)\b",
    "contiene_av": r"\b(av\.?|avenida)\b",
    "contiene_calle": r"\b(calle|cll\.)\b",
    "contiene_calzada": r"\b(calz\.?|calzada)\b",
    "contiene_km": r"\bkm\b",
    "tiene_numero": r"\d+"
}

if "DIRECCION" in df_raw.columns:
    dire = df_raw["DIRECCION"].astype(str).str.lower()
    resumen_dir = {}
    total_dir_no_vacias = dire.replace({"": np.nan}).dropna().shape[0]
    resumen_dir["total_no_vacias"] = total_dir_no_vacias

    for nombre, pat in patrones.items():
        resumen_dir[nombre] = int(dire.str.contains(pat, regex=True).sum())

    resumen_dir_df = pd.DataFrame([resumen_dir])
    resumen_dir_df.to_csv(os.path.join(OUT_DIR, "10_resumen_direcciones.csv"), index=False)
    display(resumen_dir_df.head())
else:
    print("⚠️ No existe columna DIRECCION en el CSV crudo.")


/tmp/ipython-input-4028851540.py:17: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  resumen_dir[nombre] = int(dire.str.contains(pat, regex=True).sum())


,total_no_vacias,contiene_zona,contiene_av,contiene_calle,contiene_calzada,contiene_km,tiene_numero
0,11277,5037,3085,3568,243,254,8099


In [19]:
if "ESTABLECIMIENTO" in df_raw.columns:
    est_norm = normaliza_nombre(df_raw["ESTABLECIMIENTO"])
    df_tmp = df_raw.copy()
    df_tmp["_EST_NORM"] = est_norm

    # grupos con el mismo normalizado
    grp = df_tmp.groupby("_EST_NORM").size().reset_index(name="conteo").sort_values("conteo", ascending=False)
    grp = grp[grp["conteo"] > 1]  # solo los que aparecen repetidos
    grp.to_csv(os.path.join(OUT_DIR, "11_establecimientos_norm_repetidos.csv"), index=False)

    # Muestra de los top 30 grupos repetidos
    candidatos = df_tmp[df_tmp["_EST_NORM"].isin(grp["_EST_NORM"].head(30))]
    candidatos = candidatos.sort_values("_EST_NORM")
    candidatos.to_csv(os.path.join(OUT_DIR, "12_establecimientos_posibles_variaciones.csv"), index=False)

    print("Grupos con nombre normalizado repetido (top 10):")
    display(grp.head(10))
else:
    print("⚠️ No existe columna ESTABLECIMIENTO en el CSV crudo.")


Grupos con nombre normalizado repetido (top 10):


,_EST_NORM,conteo
3996,instituto nacional de educacion diversificada,466
3586,instituto de educacion diversificada por cooperativa de ensenanza,47
142,centro de educacion extraescolar -ceex-,38
3507,instituto de computacion informatica,27
3702,instituto diversificado por cooperativa,27
3280,escuela normal de educacion fisica,24
3755,instituto guatemalteco de educacion radiofonica (iger),20
81,asociacion de maestros de educacion rural de guatemala -amerg-,19
3577,instituto de educacion diversificada por cooperativa,18
3709,instituto diversificado por cooperativa de ensenanza,16


In [18]:
resumen_crudo = {
    "filas_totales": len(df_raw),
    "columnas_totales": len(df_raw.columns),
    "duplicados_completos": int(df_raw.duplicated(keep=False).sum()),
    "columnas_con_faltantes": int((df_raw.replace({"": np.nan}).isna().sum() > 0).sum())
}

# Porcentaje de filas con TEL no válido (si existe)
if "TELEFONO" in df_raw.columns:
    tel = df_raw["TELEFONO"].astype(str)
    tel_no_vacios = tel.replace({"": np.nan}).dropna()
    tel_norm = tel_no_vacios.str.replace(r"[^0-9]", "", regex=True)
    tel_invalidos = (tel_norm.str.len() != 8).sum()
    resumen_crudo["telefonos_invalidos"] = int(tel_invalidos)
    resumen_crudo["%_telefonos_invalidos_sobre_no_vacios"] = round(100 * tel_invalidos / max(1, len(tel_no_vacios)), 2)

# Columnas con mayor % de faltantes (top 5)
falt_top5 = (df_raw.replace({"": np.nan}).isna().mean() * 100).sort_values(ascending=False).head(5).round(2)
resumen_crudo_df = pd.DataFrame([resumen_crudo])
resumen_crudo_df.to_csv(os.path.join(OUT_DIR, "13_resumen_crudo.csv"), index=False)

print("Resumen crudo:")
display(resumen_crudo_df)
print("\nTop 5 columnas con mayor % de faltantes:")
display(falt_top5.to_frame("porcentaje_faltante"))


Resumen crudo:


,filas_totales,columnas_totales,duplicados_completos,columnas_con_faltantes,telefonos_invalidos,%_telefonos_invalidos_sobre_no_vacios
0,11277,17,0,6,1900,16.85



Top 5 columnas con mayor % de faltantes:


,porcentaje_faltante
DIRECTOR,13.29
TELEFONO,8.29
SUPERVISOR,4.68
DISTRITO,4.66
DIRECCION,0.68
